In [ ]:
# Installs the HTRC feature reader
#
!pip install htrc-feature-reader
from htrc_features import FeatureReader
import os


Andy Chen

Adi Pall-Pareek

Juan F. Atehortúa

DCS 1020
# An Introspective look into the computational tools used in reading at scale

Ted Underwood Genre Project


#Bibliography
- Jacob Jett, Boris Capitanu, Deren Kudeki, Timothy Cole, Yuerong Hu, Peter Organisciak, Ted Underwood, Eleanor Dickson Koehl, Ryan Dubnicek, J. Stephen Downie (2020). The HathiTrust Research Center Extracted Features Dataset (2.0).HathiTrust Research Center.https://doi.org/10.13012/R2TE-C227